# Notebook for Exploratory Data Analysis
Notes:
 - Any general "to do" can go here

## Standard Loads for Uprooting

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import uproot as ur

print('Numpy version: {}'.format(np.__version__))
print('Uproot version: {}'.format(ur.__version__))

Numpy version: 1.21.4
Uproot version: 4.1.8


## Load Root Files and Make Histograms